In [1]:
import cudf
import cuml
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
base = cudf.read_csv('US_Accidents_March23.csv')

In [3]:
base.shape, type(base)

((7728394, 46), cudf.core.dataframe.DataFrame)

In [4]:
base.isnull().sum()

ID                             0
Source                         0
Severity                       0
Start_Time                     0
End_Time                       0
Start_Lat                      0
Start_Lng                      0
End_Lat                  3402762
End_Lng                  3402762
Distance(mi)                   0
Description                    5
Street                     10869
City                         253
County                         0
State                          0
Zipcode                     1915
Country                        0
Timezone                    7808
Airport_Code               22635
Weather_Timestamp         120228
Temperature(F)            163853
Wind_Chill(F)            1999019
Humidity(%)               174144
Pressure(in)              140679
Visibility(mi)            177098
Wind_Direction            175206
Wind_Speed(mph)           571233
Precipitation(in)        2203586
Weather_Condition         173459
Amenity                        0
Bump      

In [5]:
df = base.dropna()

In [6]:
df

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,A-3412645,Source1,3,2016-02-08 00:37:08,2016-02-08 06:37:08,40.108910,-83.092860,40.112060,-83.031870,3.230,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,A-3412650,Source1,3,2016-02-08 07:53:43,2016-02-08 13:53:43,39.172393,-84.492792,39.170476,-84.501798,0.500,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,A-3412654,Source1,2,2016-02-08 11:51:46,2016-02-08 17:51:46,41.375310,-81.820170,41.367860,-81.821740,0.521,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,A-3412656,Source1,2,2016-02-08 15:16:43,2016-02-08 21:16:43,40.109310,-82.968490,40.110780,-82.984000,0.826,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,A-3412657,Source1,2,2016-02-08 15:43:50,2016-02-08 21:43:50,39.192880,-84.477230,39.196150,-84.473350,0.307,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,A-7777757,Source1,2,2019-08-23 18:03:25,2019-08-23 18:32:01,34.002480,-117.379360,33.998880,-117.370940,0.543,...,False,False,False,False,False,False,Day,Day,Day,Day
7728390,A-7777758,Source1,2,2019-08-23 19:11:30,2019-08-23 19:38:23,32.766960,-117.148060,32.765550,-117.153630,0.338,...,False,False,False,False,False,False,Day,Day,Day,Day
7728391,A-7777759,Source1,2,2019-08-23 19:00:21,2019-08-23 19:28:49,33.775450,-117.847790,33.777400,-117.857270,0.561,...,False,False,False,False,False,False,Day,Day,Day,Day
7728392,A-7777760,Source1,2,2019-08-23 19:00:21,2019-08-23 19:29:42,33.992460,-118.403020,33.983110,-118.395650,0.772,...,False,False,False,False,False,False,Day,Day,Day,Day


In [7]:
cols_dataset = df.columns.values.tolist()
cols_dataset

['ID',
 'Source',
 'Severity',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'County',
 'State',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)',
 'Weather_Condition',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [8]:
cols = ['Severity', 'Source', 'County', 'State', 'Weather_Condition', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']
cols

['Severity',
 'Source',
 'County',
 'State',
 'Weather_Condition',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight',
 'Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop']

In [9]:
len(cols)

22

In [10]:
drop_cols = [c for c in cols_dataset if c not in cols]
drop_cols

['ID',
 'Start_Time',
 'End_Time',
 'Start_Lat',
 'Start_Lng',
 'End_Lat',
 'End_Lng',
 'Distance(mi)',
 'Description',
 'Street',
 'City',
 'Zipcode',
 'Country',
 'Timezone',
 'Airport_Code',
 'Weather_Timestamp',
 'Temperature(F)',
 'Wind_Chill(F)',
 'Humidity(%)',
 'Pressure(in)',
 'Visibility(mi)',
 'Wind_Direction',
 'Wind_Speed(mph)',
 'Precipitation(in)']

In [11]:
df = df.drop(columns=drop_cols)

In [12]:
df

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,Source1,3,Franklin,OH,Light Rain,False,False,False,False,False,...,False,False,False,False,False,False,Night,Night,Night,Night
3402767,Source1,3,Hamilton,OH,Light Rain,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
3402771,Source1,2,Cuyahoga,OH,Snow,False,False,False,False,True,...,False,False,False,False,False,False,Day,Day,Day,Day
3402773,Source1,2,Franklin,OH,Snow,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
3402774,Source1,2,Hamilton,OH,Light Snow,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,Source1,2,Riverside,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
7728390,Source1,2,San Diego,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day
7728391,Source1,2,Orange,CA,Partly Cloudy,False,False,False,False,True,...,False,False,False,False,False,False,Day,Day,Day,Day
7728392,Source1,2,Los Angeles,CA,Fair,False,False,False,False,False,...,False,False,False,False,False,False,Day,Day,Day,Day


Applying Label Encoder for some columns

In [13]:
from cuml.preprocessing import LabelEncoder

In [14]:
label_encoder = LabelEncoder()

In [15]:
for c in cols:
    if c != 'Severity':
        df[c] = label_encoder.fit_transform(df[c])

In [16]:
df

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
3402762,0,3,532,33,53,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
3402767,0,3,638,33,53,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3402771,0,2,388,33,97,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3402773,0,2,532,33,97,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3402774,0,2,638,33,61,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728389,0,2,1258,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7728390,0,2,1310,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7728391,0,2,1098,3,76,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7728392,0,2,870,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Class imbalance

In [17]:
df['Severity'].value_counts()

Severity
2    3348445
4     112511
3      68026
1      25567
Name: count, dtype: int64

In [18]:
df = df.to_pandas()

In [19]:
type(df)

pandas.core.frame.DataFrame

In [20]:
from sklearn.utils import resample

In [21]:
df_s1 = df[df['Severity'] == 1]
df_s2 = df[df['Severity'] == 2]
df_s3 = df[df['Severity'] == 3]
df_s4 = df[df['Severity'] == 4]

In [22]:
df_s1

,Source,Severity,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
6525681,0,1,1163,2,71,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0
6527285,0,1,1163,2,14,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6540036,0,1,868,33,58,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6542183,0,1,912,2,14,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6543739,0,1,1384,3,14,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7228848,0,1,1571,25,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228849,0,1,200,25,76,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228851,0,1,209,25,71,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7228852,0,1,1571,25,71,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
count = int(max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]) / 20)

/tmp/ipykernel_203181/2922344603.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  count = int(max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]) / 20)
/tmp/ipykernel_203181/2922344603.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  count = int(max(df_s1.count()[0], df_s2.count()[0], df_s3.count()[0], df_s4.count()[0]) / 20)
/tmp/ipykernel_203181/2922344603.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[p

In [24]:
count

167422

In [25]:
df_s1 = resample(df_s1, replace=df_s1.count()[0] < count, n_samples=count, random_state=42)
df_s2 = resample(df_s2, replace=df_s2.count()[0] < count, n_samples=count, random_state=42)
df_s3 = resample(df_s3, replace=df_s3.count()[0] < count, n_samples=count, random_state=42)
df_s4 = resample(df_s4, replace=df_s4.count()[0] < count, n_samples=count, random_state=42)

/tmp/ipykernel_203181/868857252.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_s1 = resample(df_s1, replace=df_s1.count()[0] < count, n_samples=count, random_state=42)
/tmp/ipykernel_203181/868857252.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_s2 = resample(df_s2, replace=df_s2.count()[0] < count, n_samples=count, random_state=42)
/tmp/ipykernel_203181/868857252.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  

In [26]:
df = pd.concat([df_s1, df_s2, df_s3, df_s4])

In [27]:
##bring back df to GPU
df = cudf.from_pandas(df)
type(df)

cudf.core.dataframe.DataFrame

In [28]:
print(df['Severity'].value_counts())

Severity
2    167422
3    167422
4    167422
1    167422
Name: count, dtype: int64


Doing oneHot encoding

In [29]:
X = df.drop('Severity', axis=1)

In [30]:
X

,Source,County,State,Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
7144412,0,465,16,76,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7087188,0,851,8,7,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6935720,0,1430,3,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6973530,0,912,2,14,0,0,0,0,0,0,...,0,0,0,0,1,0,1,1,0,0
7129626,0,912,2,14,0,0,1,0,0,0,...,0,0,0,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5945958,0,1104,32,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5668453,0,885,36,76,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5147423,0,1171,43,14,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
5567787,0,14,43,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
from cuml.preprocessing import OneHotEncoder
from cuml.compose import ColumnTransformer

In [32]:
idx = [*range(0,20)]
idx

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [33]:
onehotencoder = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(sparse=False), idx)], remainder='passthrough')

In [34]:
X = onehotencoder.fit_transform(X)

/opt/conda/lib/python3.12/site-packages/cuml/preprocessing/encoders.py:311: FutureWarning: `sparse` was renamed to `sparse_output` in version 24.06 and will be removed in 25.08. `sparse_output` is ignored unless you leave `sparse` set to its default value.
  warnings.warn(


In [35]:
X.shape

(669688, 1762)

Data Scaling

In [36]:
from cuml.preprocessing import StandardScaler
scaler_dataset = StandardScaler()

In [37]:
X = scaler_dataset.fit_transform(X)

In [38]:
X

,0,1,2,3,4,5,6,7,8,9,...,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761
0,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962
1,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,-1.944384,1.944384,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962
2,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962
3,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,-1.944384,1.944384,0.0,-1.553427,1.553427,-1.704877,1.704877,0.525820,-0.525820,-0.472962
4,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,-1.944384,1.944384,0.0,-1.553427,1.553427,-1.704877,1.704877,-1.902431,1.902431,-0.472962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669683,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962
669684,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962
669685,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,-1.553427,1.553427,-1.704877,1.704877,-1.902431,1.902431,2.114401
669686,0.0,-0.008013,-0.006804,-0.016076,-0.021454,-0.007632,-0.07979,-0.001222,-0.023067,-0.006231,...,0.514628,-0.514628,0.0,0.643524,-0.643524,0.586559,-0.586559,0.525820,-0.525820,-0.472962


In [39]:
X.shape
type(X)

cudf.core.dataframe.DataFrame

In [40]:
type(df)

cudf.core.dataframe.DataFrame

Dividing for ML training

In [41]:
from cuml.model_selection import  train_test_split


In [42]:
y = df['Severity']
y = LabelEncoder().fit_transform(y)

In [43]:
y.shape

(669688,)

In [47]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.15, random_state = 42)

In [48]:
X_treinamento.shape, X_teste.shape, y_treinamento.shape, y_teste.shape

((569235, 1762), (100453, 1762), (569235,), (100453,))

In [49]:
import pickle
with open('dataset.pkl', mode = 'wb') as f:
  pickle.dump([X_treinamento, y_treinamento, X_teste, y_teste], f)

Training using KNeighbors

In [55]:
from cuml.neighbors import KNeighborsClassifier
type(X_treinamento),type(y_treinamento), type(X_teste),type(y_teste)

(cudf.core.dataframe.DataFrame,
 cudf.core.series.Series,
 cudf.core.dataframe.DataFrame,
 cudf.core.series.Series)

In [56]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_treinamento, y_treinamento)

KNeighborsClassifier()

In [57]:
previsoes = knn.predict(X_teste)
previsoes

96530     3
584291    2
152352    1
120089    0
40597     0
         ..
569955    3
392274    2
293880    1
348297    2
63301     0
Length: 100453, dtype: uint8

In [58]:
from cuml.metrics import accuracy_score

In [59]:
accuracy_score(y_teste, previsoes)

0.6573920249938965